In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
engine = create_engine('sqlite:///emp.db')

In [3]:
# File 1 - Departments
path = "data/departments.csv"
department_df = pd.read_csv(path)
department_df.to_sql('departments', con=engine, index=True, index_label='id', if_exists='replace')
department_df

,dept_no,dept_name
0,d001,Marketing
1,d002,Finance
2,d003,Human Resources
3,d004,Production
4,d005,Development
5,d006,Quality Management
6,d007,Sales
7,d008,Research
8,d009,Customer Service


In [4]:
# File 2 - Work title
path = "data/titles.csv"
title_df = pd.read_csv(path)
title_df.to_sql('titles', con=engine, index=True, index_label='id', if_exists='replace')
title_df

,title_id,title
0,s0001,Staff
1,s0002,Senior Staff
2,e0001,Assistant Engineer
3,e0002,Engineer
4,e0003,Senior Engineer
5,e0004,Technique Leader
6,m0001,Manager


In [5]:
# File 3 - Employees Data
path = "data/employees.csv"
employee_df = pd.read_csv(path)
employee_df.to_sql('employees', con=engine, index=True, index_label='id', if_exists='replace')
employee_df.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date
0,473302,s0001,7/25/1953,Hideyuki,Zallocco,M,4/28/1990
1,475053,e0002,11/18/1954,Byong,Delgrande,F,9/7/1991
2,57444,e0002,1/30/1958,Berry,Babb,F,3/21/1992
3,421786,s0001,9/28/1957,Xiong,Verhoeff,M,11/26/1987
4,282238,e0003,10/28/1952,Abdelkader,Baumann,F,1/18/1991


In [6]:
# File 4 - Employee number relation to departments
path = "data/dept_emp.csv"
dept_emp_df = pd.read_csv(path)
dept_emp_df.to_sql('dept_emp', con=engine, index=True, index_label='id', if_exists='replace')
dept_emp_df.head()

,emp_no,dept_no
0,10001,d005
1,10002,d007
2,10003,d004
3,10004,d004
4,10005,d003


In [7]:
# File 5 - Manager
path = "data/dept_manager.csv"
dept_manager_df = pd.read_csv(path)
dept_manager_df.to_sql('dept_manager', con=engine, index=True, index_label='id', if_exists='replace')
dept_manager_df.head()

,dept_no,emp_no
0,d001,110022
1,d001,110039
2,d002,110085
3,d002,110114
4,d003,110183


In [8]:
# File 6 - Salaries
path = "data/salaries.csv"
salaries_df = pd.read_csv(path)
salaries_df.to_sql('salaries', con=engine, index=True, index_label='id', if_exists='replace')
salaries_df.head()

,emp_no,salary
0,10001,60117
1,10002,65828
2,10003,40006
3,10004,40054
4,10005,78228


In [9]:
# List the employee number, last name, first name, sex, and salary of each employee
sal_emp = pd.merge(employee_df[["emp_no","first_name","last_name","sex"]],salaries_df, on='emp_no')
sal_emp.head()                   

,emp_no,first_name,last_name,sex,salary
0,473302,Hideyuki,Zallocco,M,40000
1,475053,Byong,Delgrande,F,53422
2,57444,Berry,Babb,F,48973
3,421786,Xiong,Verhoeff,M,40000
4,282238,Abdelkader,Baumann,F,40000


In [10]:
# List the first name, last name, and hire date for the employees who were hired in 198
from datetime import datetime
employee_df["hire_date"] = pd.to_datetime(employee_df["hire_date"], format ='%m/%d/%Y')
employee_df["birth_date"] = pd.to_datetime(employee_df["birth_date"], format ='%m/%d/%Y')
employee_df.head()

,emp_no,emp_title_id,birth_date,first_name,last_name,sex,hire_date
0,473302,s0001,1953-07-25,Hideyuki,Zallocco,M,1990-04-28
1,475053,e0002,1954-11-18,Byong,Delgrande,F,1991-09-07
2,57444,e0002,1958-01-30,Berry,Babb,F,1992-03-21
3,421786,s0001,1957-09-28,Xiong,Verhoeff,M,1987-11-26
4,282238,e0003,1952-10-28,Abdelkader,Baumann,F,1991-01-18


In [21]:
hire_1986 = employee_df.loc[employee_df["hire_date"].dt.strftime('%Y') == '1986',
                            ["first_name","last_name","hire_date"]]
hire_1986.head()

,first_name,last_name,hire_date
5,Eran,Cusworth,1986-11-14
14,Bojan,Zallocco,1986-10-14
19,Nevio,Demizu,1986-05-18
29,Ziva,Vecchi,1986-07-03
33,Mohit,Speek,1986-01-14


In [26]:
# List the manager of each department along with their department number, department name, employee number, last name, and first name 
manager_det = pd.merge(dept_manager_df, department_df, on = "dept_no")
manager_emp = pd.merge(manager_det, employee_df[["emp_no","first_name","last_name"]], on = "emp_no")
manager_emp.head()

,dept_no,emp_no,dept_name,first_name,last_name
0,d001,110022,Marketing,Margareta,Markovitch
1,d001,110039,Marketing,Vishwani,Minakawa
2,d002,110085,Finance,Ebru,Alpin
3,d002,110114,Finance,Isamu,Legleitner
4,d003,110183,Human Resources,Shirish,Ossenbruggen


In [28]:
# List the department number for each employee along with that employee’s employee number, last name, first name, and department name
emp_dept = pd.merge(dept_emp_df, employee_df[["emp_no","first_name","last_name"]], on="emp_no")
emp_dept_name = pd.merge(department_df, emp_dept, on="dept_no")
emp_dept_name.head()

,dept_no,dept_name,emp_no,first_name,last_name
0,d001,Marketing,10017,Cristinel,Bouloucos
1,d001,Marketing,10055,Georgy,Dredge
2,d001,Marketing,10058,Berhard,McFarlin
3,d001,Marketing,10108,Lunjin,Giveon
4,d001,Marketing,10140,Yucel,Auria


In [32]:
# List first name, last name, and sex of each employee whose first name is Hercules and whose last name begins with the letter B
H_emp = employee_df.loc[employee_df["first_name"] == "Hercules",["first_name","last_name","sex"]]
B_emp = H_emp[H_emp.last_name.str[0] == 'B']
B_emp.head()

,first_name,last_name,sex
4218,Hercules,Baer,M
15577,Hercules,Biron,F
21897,Hercules,Birge,F
30877,Hercules,Berstel,F
45926,Hercules,Bernatsky,M


In [35]:
# List each employee in the Sales department, including their employee number, last name, and first name
sales = emp_dept_name.loc[emp_dept_name["dept_name"] == "Sales",["dept_name","emp_no","first_name","last_name"]]
sales.head()

,dept_name,emp_no,first_name,last_name
234652,Sales,10002,Bezalel,Simmel
234653,Sales,10016,Kazuhito,Cappelletti
234654,Sales,10034,Bader,Swan
234655,Sales,10041,Uri,Lenart
234656,Sales,10050,Yinghua,Dredge


In [38]:
# List each employee in the Sales and Development departments, including their employee number, last name, first name, and department name
sales_dept = emp_dept_name.loc[(emp_dept_name["dept_name"] == "Sales") | (emp_dept_name["dept_name"] == "Development")
                               ,["dept_name","emp_no","first_name","last_name"]]
sales_dept

,dept_name,emp_no,first_name,last_name
128828,Development,10001,Georgi,Facello
128829,Development,10006,Anneke,Preusig
128830,Development,10008,Saniya,Kalloufi
128831,Development,10012,Patricio,Bridgland
128832,Development,10014,Berni,Genin
...,...,...,...,...
286892,Sales,499976,Guozhong,Felder
286893,Sales,499980,Gino,Usery
286894,Sales,499986,Nathan,Ranta
286895,Sales,499987,Rimli,Dusink


In [58]:
# List the frequency counts, in descending order, of all the employee last names (that is, how many employees share each last name) 
dups_values = employee_df.pivot_table(columns=['last_name'], aggfunc='size')
dups_values.sort_values(ascending=False)

last_name
Baba          226
Coorg         223
Gelosh        223
Sudbeck       222
Farris        222
             ... 
Guardalben    148
Zykh          148
Merro         147
Sadowsky      145
Foolsday        1
Length: 1638, dtype: int64